# Converting NeXus to ASCII Files
### Jan 2023

Using [nexus2srs](https://github.com/DanPorter/nexus2srs)

In [1]:
import sys, os
import numpy as np
import datetime
from nexus2srs import nxs2dat, module_info

files = [
    "example_files/782761.nxs",  # hkl scan
    "example_files/794932.nxs",  # eta scan
    "example_files/815893.nxs",  # NXclassic_scan (eta)
    "example_files/879486.nxs",  # 2D scan
    "example_files/970792.nxs",  # kdelta scan, missing measurement, scan length < 6
]

print(module_info())

Python version 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
 at: C:\ProgramData\Anaconda3\python.exe
 nexus2srs version 0.5.1 (2023/01/25): C:\Users\grp66007\OneDrive - Diamond Light Source Ltd\PythonProjects\nexus2srs\nexus2srs\__init__.py
     numpy version: 1.21.2
      h5py version: 2.9.0
Running in directory: C:\Users\grp66007\OneDrive - Diamond Light Source Ltd\PythonProjects\nexus2srs



In [2]:
# Convert NeXus to SRS
nxs2dat("example_files/782761.nxs", "example_files/782761_new.dat", True)

Nexus File: example_files/782761.nxs
NXmeasurement - uses arrays in measurement group
image addresses: []
Scan length: 41
Written to: example_files/782761_new.dat


In [3]:
# Check new File:
with open("example_files/782761_new.dat") as f:
    print(f.read())

 &SRS
 SRSRUN=782761,SRSDAT=20190711,SRSTIM=083632,
 SRSSTN='BASE',SRSPRJ='GDA_BASE',SRSEXP='Emulator',
 SRSTLE='                                                            ',
 SRSCN1='        ',SRSCN2='        ',SRSCN3='        ',
<MetaDataAtStart>
cmd='scan hkl [0, 0.5, -16.5] [0, 0.9, -16.5] [0, 0.01, 0] pil3_100k 5 roi1 roi2 Ta'
date='Thu Jul 11 08:36:32 2019'
pilatus3_100k_path_template='782761-pilatus3_100k-files/%05d.tif'
ppchi=-45.000245748
pppiezo1=0.0
pppiezo2=0.0
ppth1=-2.0001154498
ppth2=-14.9997751982
ppz1=11.0
ppz2=22.0
alpha=32.860270453797625
s1xcentre=-0.003
s1xgap=2.5
s1ycentre=0.0015
s1ygap=0.999
s2xcentre=0.0
s2xgap=19.99915
s2ycentre=0.00025
s2ygap=19.9995
s3xcentre=4.4605
s3xgap=25.003
s3ycentre=-0.6825
s3ygap=20.013
s4xcentre=-6.00275
s4xgap=32.9859
s4ycentre=0.0035
s4ygap=26.014
shtr3x=12.425
shtr3y=4.24
beta=-32.85909453734413
delta_offset=0.0
dettrans=0.0
chi=2.73633690142
delta=77.5249817791
delta_axis_offset=8.8
en=14.0000043916
eta=38.7624716254
gam=-7.5845

In [4]:
# Load file using Py16progs, compare vs old scan file
pth = os.path.expanduser('~/OneDrive - Diamond Light Source Ltd/PythonProjects')
sys.path.insert(0, pth + '/Py16')
from Py16progs import read_dat_file


In [5]:
dold = read_dat_file("example_files/782761.dat")
dnew = read_dat_file("example_files/782761_new.dat")

mold = dold.metadata
mnew = dnew.metadata

print('\n--- Compare %s files ---' % "782761.dat")
print('Scan Items the same: %s' % (dold.keys() == dnew.keys()))
print('Scan Order the same: %s' % (list(dold.keys()) == list(dnew.keys())))

scan_length_old = len(next(iter(dold.values())))
scan_length_new = len(next(iter(dnew.values())))
print('Scan length the same: (%d, %d) %s' % (scan_length_old, scan_length_new, scan_length_old == scan_length_new))

print('Meta Items the same: (%d, %d) %s' % (len(mold), len(mnew), mold.keys() == mnew.keys()))
print('\nMeta Items in Old File:')
print('\t '.join(['%12s'] * 3) % ('Name', 'Old', 'New'))
for name in mold:
    newval = mnew[name] if name in mnew else '***None***'
    print('\t '.join(['%12s'] * 3) % (name, mold[name], newval))

print('\nNew Meta Items in New File:')
print('Name        Value')
for name in mnew:
    if name in mold:
        continue
    print('%12s %12s' % (name, mnew[name]))


--- Compare 782761.dat files ---
Scan Items the same: True
Scan Order the same: False
Scan length the same: (41, 41) True
Meta Items the same: (157, 176) False

Meta Items in Old File:
        Name	          Old	          New
      SRSRUN	       782761	       782761
      SRSDAT	     20190711	     20190711
      SRSTIM	       083653	       083632
      SRSSTN	         BASE	         BASE
      SRSPRJ	     GDA_BASE	     GDA_BASE
      SRSEXP	     Emulator	     Emulator
      SRSTLE	                                                             	                                                             
      SRSCN1	             	             
      SRSCN2	             	             
      SRSCN3	             	             
         cmd	 scan hkl [0, 0.5, -16.5] [0, 0.9, -16.5] [0.0, 0.01, 0.0] pil3_100k 5 roi1 roi2 Ta	 scan hkl [0, 0.5, -16.5] [0, 0.9, -16.5] [0, 0.01, 0] pil3_100k 5 roi1 roi2 Ta
        date	 Thu Jul 11 08:36:32 2019	 Thu Jul 11 08:36:32 2019
        kgam	            

In [6]:
# Check I10 datafile
nexus_dir = os.path.expanduser('~/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format')
fnxs = nexus_dir + '/I10_nexus/i10-578596.nxs'

print('Using nxs2dat')
nxs2dat(fnxs, "example_files/i10-578596.dat")

print('\n\n --- example_files/i10-578596.dat ---')
with open("example_files/i10-578596.dat") as f:
    print(f.read())

Using nxs2dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I10_nexus/i10-578596.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (351,)
Scan length: 351
Written to: example_files/i10-578596.dat


 --- example_files/i10-578596.dat ---
 &SRS
 SRSRUN=578596,SRSDAT=20191122,SRSTIM=180400,
 SRSSTN='BASE',SRSPRJ='GDA_BASE',SRSEXP='Emulator',
 SRSTLE='                                                            ',
 SRSCN1='        ',SRSCN2='        ',SRSCN3='        ',
<MetaDataAtStart>
cmd='scan egy_g_idu_circ_pos_energy 520 555 0.1 mcsr16_g 0.1989144200709216 mcsr17_g 0.1989144200709216 mcsr18_g 0.1989144200709216 mcsr19_g 0.1989144200709216'
date='Fri Nov 22 18:04:00 2019'
cernox=113.7494
Cmes=2.102194e-06
Cset=0.0
Pmes=1.5008187317618e-09
Rmes=339.61171043205337
Vmes=0.0007139297
idd_gap=200.001
idd_jawphase=-0.368
idd_rowphase1=0.0
idd_rowphase2=0.0
idd_rowphase3=0.0
idd_rowphase4=0.0
idd_sepphase=0.0
idu_gap=18.102

In [7]:
# Check I21 datafile using nxs2dat2
nexus_dir = os.path.expanduser('~/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format')
fnxs = nexus_dir + '/I21_example/examples/Proposal_ID/i21-157111.nxs'

print('Using nxs2dat')
nxs2dat(fnxs, "example_files/i21-157111.dat")

print('\n\n --- example_files/i21-157111.dat ---')
with open("example_files/i21-157111.dat") as f:
    print(f.read())

Using nxs2dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID/i21-157111.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan length: 0
Written to: example_files/i21-157111.dat


 --- example_files/i21-157111.dat ---
 &SRS
 SRSRUN=157111,SRSDAT=20210112,SRSTIM=162959,
 SRSSTN='BASE',SRSPRJ='GDA_BASE',SRSEXP='Emulator',
 SRSTLE='                                                            ',
 SRSCN1='        ',SRSCN2='        ',SRSCN3='        ',
<MetaDataAtStart>
cmd='scan ds 1 1 1 xcam 180 m4c1 180 checkbeam'
date='Tue Jan 12 16:29:59 2021'
cff=3.335
chi=0.0
difftth=-0.000361432
draincurrent=-0.0002413988257288233
energy=927.6474
epics_armtth=149.9997
fastshutter_x=29.15
idgap=28.5192
bottomInner=28.001
bottomOuter=-0.001
enabled=1.0
gap=28.5192
mode='[b'Low Noise']'
polarisation='LV'
rowPhase=27.999000000000002
topInner=0.002
topOuter=27.999000000000002
cryostat=24.9973
demand=25.0
heater=14.9

In [8]:
# Function to convert and compare NeXus Files
import tempfile
TMPDIR = tempfile.gettempdir()

def nxcompare(nxfile):
        filename, _ = os.path.splitext(nxfile)
        filepath, name = os.path.split(filename)
        oldfile = filename + '.dat'
        # newfile = os.path.join(TMPDIR, name + '.dat')
        newfile = os.path.join(TMPDIR, 'nxcompare.dat')
        # Convert file
        nxs2dat(nxfile, newfile)
        
        # Read New file
        dnew = read_dat_file(newfile)
        n_scan_new = len(dnew)
        scan_length_new = len(next(iter(dnew.values()))) if dnew else 0
        n_meta_new = len(dnew.metadata)
        
        n_scan_old = '-'
        scan_length_old = '-'
        n_meta_old = '-'
        # Read old file if it exists
        if os.path.isfile(oldfile):
            try:
                dold = read_dat_file(oldfile)
                n_scan_old = len(dold)
                scan_length_old = len(next(iter(dold.values())))
                n_meta_old = len(dold.metadata)
            except ValueError:
                pass
        
        out = '%10s scan %4s (%4s) scan_len %4s (%4s) meta %4s (%4s)' % (name, n_scan_new, n_scan_old, scan_length_new, scan_length_old, n_meta_new, n_meta_old)
        return out
    

In [9]:
# Test files in local Nexus_Format Directory
nexus_dir = os.path.expanduser('~/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus')

files = [os.path.join(nexus_dir, file) for file in os.listdir(nexus_dir) if file.endswith('.nxs')]

t0 = datetime.datetime.now()
run = 0
output = []
failed = []
for file in files:
    run += 1
    try:
        output += [nxcompare(file)]
    except Exception as e:
        failed += [(file, e)]
        output += ['Failed on: %s' % file]
t1 = datetime.datetime.now()

print('\n\n Comparison:')
print('\n'.join(output[:50]))
print('\n'.join(output[-50:]))
print('\n\n Failures:')
print('\n'.join(['%s\n%s\n' % (f,e) for f,e in failed]))
print('\n\n Completed %d conversions with %d failures' % (run, len(failed)))
dtime = t1 - t0
print('Run time: %s (%.3f s per scan)' % (dtime, dtime.total_seconds()/ run))

Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\705061.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (5, 11)
Scan length: 55
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\782761.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\794932.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 61
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\794933.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 61
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\U

Scan length: 261
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\872548.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 45
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\872996.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 31
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\877619.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 101
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/example_nexus\877992.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 51
Written to: C:\Users\grp66007\AppData\

In [10]:
# Test files in I10 Directory
nexus_dir = os.path.expanduser('~/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I10_nexus')

files = [os.path.join(nexus_dir, file) for file in os.listdir(nexus_dir) if file.endswith('.nxs')]

t0 = datetime.datetime.now()
run = 0
output = []
failed = []
for file in files:
    run += 1
    try:
        output += [nxcompare(file)]
    except Exception as e:
        failed += [(file, e)]
        output += ['Failed on: %s' % file]
t1 = datetime.datetime.now()

print('\n\n Comparison:')
print('\n'.join(output[:50]))
print('\n'.join(output[-50:]))
print('\n\n Failures:')
print('\n'.join(['%s\n%s\n' % (f,e) for f,e in failed]))
print('\n\n Completed %d conversions with %d failures' % (run, len(failed)))
dtime = t1 - t0
print('Run time: %s (%.3f s per scan)' % (dtime, dtime.total_seconds()/ run))

Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I10_nexus\i10-578596.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (351,)
Scan length: 351
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I10_nexus\i10-578621.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (351,)
Scan length: 351
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I10_nexus\i10-578623.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (351,)
Scan length: 351
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I10_nexus\i10-578625.nxs
Generic_scan - scan field names not preserved. Search for scan_s

In [11]:
# Test files in I21 Directory
nexus_dir = os.path.expanduser('~/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID')

files = [os.path.join(nexus_dir, file) for file in os.listdir(nexus_dir) if file.endswith('.nxs')]

t0 = datetime.datetime.now()
run = 0
output = []
failed = []
for file in files:
    run += 1
    try:
        output += [nxcompare(file)]
    except Exception as e:
        failed += [(file, e)]
        output += ['Failed on: %s' % file]
t1 = datetime.datetime.now()

print('\n\n Comparison:')
print('\n'.join(output[:50]))
print('\n'.join(output[-50:]))
print('\n\n Failures:')
print('\n'.join(['%s\n%s\n' % (f,e) for f,e in failed]))
print('\n\n Completed %d conversions with %d failures' % (run, len(failed)))
dtime = t1 - t0
print('Run time: %s (%.3f s per scan)' % (dtime, dtime.total_seconds()/ run))

Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID\i21-157111.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan length: 0
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID\i21-157116.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (2,)
Scan length: 2

C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/PythonProjects/Py16\Py16progs.py:352: UserWarning: loadtxt: Empty input file: "['\n']"
  vals = np.loadtxt(lines[lineno+1:],ndmin=2)



Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID\i21-157117.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan length: 0
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID\i21-157118.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (2,)
Scan length: 2
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: C:\Users\grp66007/OneDrive - Diamond Light Source Ltd/I16/Nexus_Format/I21_example/examples/Proposal_ID\i21-157119.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan length: 0
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat


 Comparison:
i21-157111 scan    0 (   -) scan_len    0 (   -) meta  154 (   -)
i21-

In [12]:
# Test a very large number of NeXus files on the network
import datetime
datadir = r'\\data.diamond.ac.uk\i16\data\2023'
# files = []
# for root, dirs, dirfiles in os.walk(datadir):
#     for file in dirfiles:
#         if file.endswith(".nxs"):
#              files += [os.path.join(root, file)]

#print('\n'.join(files))

In [13]:
with open('i16_datafiles.txt') as f:
    files = f.read().splitlines() 

t0 = datetime.datetime.now()
run = 0
output = []
failed = []
for file in files[:1000] + files[-1000:]:
    run += 1
    try:
        output += [nxcompare(file)]
    except Exception as e:
        failed += [(file, e)]
        output += ['Failed on: %s' % file]
t1 = datetime.datetime.now()

Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\713408.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (51,)
Scan length: 51
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714441.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (21,)
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714383.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (401,)
Scan length: 401
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714158.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (21,)
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\71384

Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (32,)
Scan length: 32
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\713517.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (21,)
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714991.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (32,)
Scan length: 32
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714960.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (32,)
Scan length: 32
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\713726.nxs
Generic_scan - scan field names not preserved. Search for scan_s

Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714305.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (41,)
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\713910.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (31,)
Scan length: 31
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714788.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (28,)
Scan length: 28
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714238.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (31,)
Scan length: 31
Written to: C:\Users\grp66007\AppData\Local\Temp

Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (31,)
Scan length: 31
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714881.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (32,)
Scan length: 32
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714643.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (32,)
Scan length: 32
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714223.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (21,)
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\713605.nxs
Generic_scan - scan field names not preserved. Search for scan_s

Scan length: 2
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\713420.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (51,)
Scan length: 51
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714206.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (31,)
Scan length: 31
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\714121.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (81,)
Scan length: 81
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2018\mt18798-1\715017.nxs
Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (32,)
Scan length: 32
Written to: C:\Users\grp66007\AppData\Local\Temp\

...

NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971498.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971856.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971225.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 81
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971879.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\970908.nxs
NXmeasurement - uses

NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971386.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971655.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\970887.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 401
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971883.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971531.nxs
NXmeasurement - use

NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971994.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971385.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971723.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971696.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971159.nxs
NXmeasurement - uses

Generic_scan - scan field names not preserved. Search for scan_shape
Scan shape: (35,)
Scan length: 35
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971968.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971171.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\970970.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971909.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 41
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm3

NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971574.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat
Nexus File: \\data.diamond.ac.uk\i16\data\2023\cm33911-1\971145.nxs
NXmeasurement - uses arrays in measurement group
Scan length: 21
Written to: C:\Users\grp66007\AppData\Local\Temp\nxcompare.dat


In [14]:
print('\n\n Comparison:')
print('\n'.join(output[:50]))
print('\n'.join(output[-50:]))
print('\n\n Failures:')
print('\n'.join(['%s\n%s\n' % (f,e) for f,e in failed]))
print('\n\n Completed %d conversions with %d failures' % (run, len(failed)))
dtime = t1 - t0
print('Run time: %s (%.3f s per scan)' % (dtime, dtime.total_seconds()/ run))



 Comparison:
    713408 scan   39 (  32) scan_len   51 (  51) meta  180 ( 155)
    714441 scan   36 (  29) scan_len   21 (  21) meta  182 ( 155)
    714383 scan   35 (  28) scan_len  401 ( 401) meta  180 ( 155)
    714158 scan   36 (  29) scan_len   21 (  21) meta  182 ( 155)
    713843 scan   21 (  14) scan_len   22 (  22) meta  169 ( 154)
    714856 scan   38 (  31) scan_len   32 (  32) meta  180 ( 155)
    713452 scan   33 (  26) scan_len    9 (   9) meta  180 ( 155)
    714860 scan   38 (  31) scan_len   32 (  32) meta  180 ( 155)
    714233 scan   40 (  33) scan_len   51 (  51) meta  180 ( 155)
    715064 scan   38 (  31) scan_len   32 (  32) meta  180 ( 155)
    714465 scan   36 (  29) scan_len   21 (  21) meta  182 ( 155)
    713782 scan   21 (  14) scan_len   27 (  27) meta  169 ( 154)
    714056 scan   36 (  29) scan_len   31 (  31) meta  180 ( 155)
    714918 scan   38 (  31) scan_len   32 (  32) meta  180 ( 155)
    713461 scan   33 (  26) scan_len   21 (  21) meta  182 ( 

In [15]:
# Compare with the old style with hard-coded commands
from nexus2srs.nexus2srs_old import nxs2dat as nxs2dat_old
def nxcompare2(nxfile):
        filename, _ = os.path.splitext(nxfile)
        filepath, name = os.path.split(filename)
        oldfile = filename + '.dat'
        # newfile = os.path.join(TMPDIR, name + '.dat')
        newfile = os.path.join(TMPDIR, 'nxcompare.dat')
        # Convert file
        nxs2dat_old(nxfile, newfile)
        
        # Read New file
        dnew = read_dat_file(newfile)
        n_scan_new = len(dnew)
        scan_length_new = len(next(iter(dnew.values()))) if dnew else 0
        n_meta_new = len(dnew.metadata)
        
        n_scan_old = '-'
        scan_length_old = '-'
        n_meta_old = '-'
        # Read old file if it exists
        if os.path.isfile(oldfile):
            try:
                dold = read_dat_file(oldfile)
                n_scan_old = len(dold)
                scan_length_old = len(next(iter(dold.values())))
                n_meta_old = len(dold.metadata)
            except ValueError:
                pass
        
        out = '%10s scan %4s (%4s) scan_len %4s (%4s) meta %4s (%4s)' % (name, n_scan_new, n_scan_old, scan_length_new, scan_length_old, n_meta_new, n_meta_old)
        return out


# t0 = datetime.datetime.now()
# run = 0
# output = []
# failed = []
# for file in files[-100:]:
#     run += 1
#     try:
#         output += [nxcompare2(file)]
#     except Exception as e:
#         failed += [(file, e)]
#         output += ['Failed on: %s' % file]
# t1 = datetime.datetime.now()

In [16]:
# print('\n\n Comparison (hard-coded style):')
# print('\n'.join(output[:50]))
# print('\n'.join(output[-50:]))
# print('\n\n Failures:')
# print('\n'.join(['%s\n%s\n' % (f,e) for f,e in failed]))
# print('\n\n Completed %d conversions with %d failures' % (run, len(failed)))
# dtime = t1 - t0
# print('Run time: %s (%.3f s per scan)' % (dtime, dtime.total_seconds()/ run))

In [17]:
# Specific problem case - auto-determine scan length finds unit_cell (6) rather than scan length (5)
# print(nxcompare(r"\\data.diamond.ac.uk\i16\data\2023\cm33911-1\970792.nxs"))

In [18]:
# Reload module

# from importlib import reload
# import nexus2srs
# reload(nexus2srs)
# reload(nexus2srs.nexus2srs)
# del nxs2dat
# from nexus2srs import nxs2dat
# # from nexus2srs.nexus2srs import nxs2dat2